In [58]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
import lightgbm as lgb 

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV

In [147]:
# Prep data
%run 'data-prep.ipynb'

In [116]:
print(train_features.shape)
print(train_outcome.shape)
print(test_features.shape)

(8523, 36)
(8523,)
(5681, 36)


# Model Training

In [65]:
# Below are just identifiers which we dont want to fit.
IDcols = ['Item_Identifier', 'Outlet_Identifier']
IDrows = train_

In [168]:
def train_model(clf, param_grid, train_features, train_outcome, IDcols):
    
    train_predictors = train_features.drop(IDcols, axis=1)
    
    grid_search = GridSearchCV(
        clf,
        param_grid,
        n_jobs=-1
    )
    grid_search.fit(train_predictors, train_outcome.round())
    
    print('BEST SCORE', grid_search.best_score_)
    print('BEST PARAM', grid_search.best_params_)
    
    return grid_search.best_estimator_

## XGBoost

In [170]:
xgb_param_grid = {}
# xgb_param_grid = {'xgbclassifier__n_estimators': np.arange(150, 200)}

xgb_model = train_model(
    xgb.XGBClassifier(),
    xgb_param_grid,
    train_features, 
    train_outcome, 
    IDcols
)

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
Process ForkPoolWorker-125:
Process ForkPoolWorker-128:
Process ForkPoolWorker-126:
Process ForkPoolWorker-124:
Process ForkPoolWorker-127:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.4_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.4_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.4_1/Frameworks/Python.framework/Versions/3.

KeyboardInterrupt: 

## Random Forest

In [169]:
rf_param_grid = {}

rf_model = train_model(
    RandomForestClassifier(),
    rf_param_grid,
    train_features, 
    train_outcome, 
    IDcols
)

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


BEST SCORE 0.004341194415112049
BEST PARAM {}


# Submission

In [148]:
# This cell only needs to be run once.
# Running multiple times will cause unexpected error.
test_predictors = test_features.drop(IDcols, axis=1)

In [161]:
def submit(test_features, test_predictors, model, filename):
    preds = rf_model.predict(test_predictors)
    test_features['Item_Outlet_Sales'] = preds
    submission = test_features[['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales']]
    filename = 'submission-' + filename + '.csv'
    submission.to_csv(filename, header=True)

In [ ]:
# XGBoost Submission
submit(test_features, test_predictors, xgb_model, 'xgboost')

In [162]:
# Random Forest Submission
submit(test_features, test_predictors, rf_model, 'random-forest')